# Data Generation

## Import Packages

In [144]:
import numpy as np
import scipy
from scipy.stats import norm
import pandapower as pp
import pandas as pd
import pandapower.networks
import tensorflow as tf
import csv
import torch

## Input Data

In [226]:
case = pp.networks.case9()
number_of_bus = case.bus.shape[0]
baseMVA = 100

number_of_data = 100       # Including data for both Training and Testing
number_of_training_data = 80
number_of_test_data = np.subtract(number_of_data,number_of_training_data)

load = case.load.iloc[:,6:8]
load_bus_number = case.load.iloc[:,0]

Pgenerators = np.zeros((number_of_bus,1))
Pgenerators[case.gen.iloc[:,0],0] = -case.gen.iloc[:,4]

PQload = np.zeros((number_of_bus,2))
PQload[case.load.iloc[:,0],:] = case.load.iloc[:,6:8]



### NB FB TB R X PL QL PG

Data_form = np.hstack(((np.array(case.bus_geodata.iloc[:,0:2])),\
                       PQload,Pgenerators))

## Roulette Wheel Algorithm Using Normal-PDF to Generate Scenarios

In [227]:
Scenario = np.zeros([number_of_data,Data_form.shape[0],Data_form.shape[1]])

for i in range(number_of_data):
    
    Scenario[i,:,:] = Data_form
    Scenario[i,load_bus_number,2:4] = norm.ppf(np.random.rand(load.shape[0],1),\
                                                              load,np.multiply(0.1,load))

Training_Data = Scenario[0:number_of_training_data,:,:]
Test_Data = Scenario[number_of_training_data:,:,:]




## Implement Power Flow to Get Labels

In [228]:
Label = np.zeros((number_of_data,number_of_bus,2))

for i in range(number_of_data):
    
    case.load.iloc[:,6:8] = Scenario[i,case.load.iloc[:,0],2:4]
    
    pp.runpp(case, algorithm='nr', calculate_voltage_angles='auto', init='auto',\
                  max_iteration='auto', tolerance_mva=1e-08, trafo_model='t',\
                  trafo_loading='current', enforce_q_lims=False,\
                  check_connectivity=True, voltage_depend_loads=True,\
                 consider_line_temperature=False)
    
    Label[i,:,:] = case.res_bus.iloc[:,0:2]
    
Training_Label = Label[0:number_of_training_data,:,:]
Test_Label = Label[number_of_training_data:,:,:]


## Export Data

In [243]:
#Training_Data = pd.Series(Training_Data)
#Training_Label = pd.Series(Training_Label)
#Test_Data = pd.Series(Test_Data)
#Test_Label = pd.Series(Test_Label)

Final_Data = pd.DataFrame(pd.Series([Training_Data,Training_Label]))
Final_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/Data.csv")



In [244]:
a,b = pd.read_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/Data.csv")

In [245]:
a

'Unnamed: 0'

In [246]:
b

'0'